**кластеризатор текстов**

SnowballStemmer + TFIDF + DBSCAN

_Евгений Борисов <esborisov@sevsu.ru>_

In [1]:
import pandas as pd
from tqdm.auto import tqdm

pd.options.display.precision = 2 
pd.options.display.max_colwidth = 200 
tqdm.pandas()

## загрузка данных

датасет news.pkl.gz   
https://disk.yandex.ru/d/8_T_XITkZ4gKAw

In [2]:
# загружаем тексты
data = pd.read_pickle('news.pkl.gz')
display( len(data) )
display(  data.sample(2) )

3196

,text,tag
2681,"Ученые решили установить, почему в организме человека образовываются раковые клетки и как они себя ведут.\n\nБлагодаря очередному исследованию ученым удалось установить, как связан жир в организме...",science
3138,"Трибуна (Сыктывкар) Схватили за язык. Слово чиновника не воробей Комиссия\nадминистрации Главы и правительства Коми признала, что сотрудник администрации\nРоман Квашнев нарушил законодательство в ...",incident


In [3]:
display( len( data.drop_duplicates('text') ) )

3196

## токенайзер

In [4]:
# NLTK package manager
# import nltk
# nltk.download()

In [5]:
# import re
# from nltk.tokenize import word_tokenize as nltk_tokenize_word

# def tokenizer(text):
#     return [
#             t for t in nltk_tokenize_word( # разбиваем текст на слова
#                 re.sub(r'</?[a-z]+>',' ',text), # удаляем xml tag 
#                 language='russian'
#             ) 
#         ]

In [6]:
# import re
# from nltk.tokenize import word_tokenize as nltk_tokenize_word
# from nltk.corpus import stopwords as nltk_stopwords

# stopwords = set(nltk_stopwords.words('russian'))

# def tokenizer(text,stopwords=stopwords):
#     return [
#             t for t in nltk_tokenize_word( # разбиваем текст на слова
#                 re.sub(r'</?[a-z]+>',' ',text), # удаляем xml tag 
#                 language='russian'
#             ) 
#             if not (
#                False
#                or (len(t)<3) # выкидываем очень короткие слова
#                or re.match(r'^[^a-zA-ZЁёА-я]+$', t) # выкидываем токены не содержащие букв
#                or re.match(r'^(\w)\1+$', t)  # выкидываем токены из одного повторяющегося символа
#                or re.match(r'^[^a-zA-ZЁёА-я].*$', t)  # выкидываем токены начинающиеся не с буквы
#                or (t in stopwords) # выкидываем предлоги, союзы и т.п.    
#             )
#         ] 

In [7]:
import re
# from razdel import sentenize
from razdel import tokenize
from nltk.corpus import stopwords as nltk_stopwords
stopwords = set(nltk_stopwords.words('russian'))

def tokenizer(text,stopwords=stopwords):
    return [
            t.text for t in tokenize( # разбиваем текст на слова
                re.sub(r'</?[a-z]+>',' ',text), # удаляем xml tag 
            ) 
            if not (
               False
               or (len(t.text)<3) # выкидываем очень короткие слова
               or re.match(r'^[^a-zA-ZЁёА-я]+$', t.text) # выкидываем токены не содержащие букв
               or re.match(r'^(\w)\1+$', t.text)  # выкидываем токены из одного повторяющегося символа
               or re.match(r'^[^a-zA-ZЁёА-я].*$', t.text)  # выкидываем токены начинающиеся не с буквы
               or (t.text in stopwords) # выкидываем предлоги, союзы и т.п.    
            )
        ] 

## выполняем частотный анализ

In [8]:
# from sklearn.feature_extraction.text import CountVectorizer
# tf_model = CountVectorizer(
#         min_df=.01, # выкидываем очень редкие слова
#         max_df=.25, # выкидываем очень частые слова
#         tokenizer=tokenizer, # ф-ция токенайзер
#         token_pattern=None, # отключаем дефолтный токенайзер
#         binary=True,
#     )

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_model = TfidfVectorizer(
        min_df=.01, # выкидываем очень редкие слова
        max_df=.10, # выкидываем очень частые слова
        use_idf=False, # не используем обратную частоту
        norm='l2', # нормируем TF
        tokenizer=tokenizer, # ф-ция токенайзер
        token_pattern=None, # отключаем дефолтный токенайзер
        ngram_range = (2,2)
    )

In [10]:
%%time

data_tf = tf_model.fit_transform( data['text'].str.lower() )

display(data_tf.shape)

(3196, 116)

CPU times: user 4.31 s, sys: 31.1 ms, total: 4.34 s
Wall time: 4.34 s


In [11]:
vocab = sorted(tf_model.vocabulary_)
display(len(vocab))
display(vocab)

116

['adobe flash',
 'afisha tut',
 'auto tut',
 'finance tut',
 'flash player',
 'html установлена',
 'javascript ваш',
 'realty tut',
 'sport tut',
 'wall street',
 'ближайшее время',
 'большая часть',
 'браузер поддерживает',
 'ваш браузер',
 'версия проигрывателя',
 'владимир путин',
 'внимание отключен',
 'возбуждено уголовное',
 'вторник декабря',
 'второе место',
 'глава государства',
 'главы государства',
 'говорится сообщении',
 'года назад',
 'данный момент',
 'дек риа',
 'декабря auto',
 'декабря lenta',
 'декабря sport',
 'декабря tut',
 'декабря года',
 'декабря обновлено',
 'декабря тасс',
 'дональд трамп',
 'дональда трампа',
 'друг друга',
 'избранного президента',
 'избранный президент',
 'иностранных дел',
 'конца года',
 'коренных малочисленных',
 'лет назад',
 'лиги чемпионов',
 'лошадиных сил',
 'малочисленных народов',
 'мвд россии',
 'миллиона рублей',
 'миллионов рублей',
 'млн долларов',
 'млн рублей',
 'москва дек',
 'москва декабря',
 'народов севера',
 'настояще

## кластеризируем

In [12]:
# оценки расстояний 
from sklearn.metrics.pairwise import euclidean_distances
d = euclidean_distances(data_tf)

display( d[d>0.].min(),d[d>0.].mean(),d.max(), )

0.018299992189866887

1.2168617334102432

1.4142135623730954

In [13]:
%%time

from sklearn.cluster import DBSCAN

data['cluster_id'] = DBSCAN(eps=.7,min_samples=3).fit(data_tf).labels_

display( data['cluster_id'].drop_duplicates().count() )

83

CPU times: user 83.5 ms, sys: 55.8 ms, total: 139 ms
Wall time: 170 ms


In [14]:
# номер кластера, количество объектов, метки объектов
# (cluster=-1 - некластеризованные DBSCAN объекты) 
cluster_descr = pd.concat([
        data[['cluster_id','tag']].groupby(['cluster_id'])['tag'].count(),
        data[['cluster_id','tag']].groupby(['cluster_id'])['tag'].apply(lambda s: set(s)).apply(' '.join)
    ],axis=1).reset_index()

cluster_descr.columns = ['cluster_id','count','tags']

display( cluster_descr )

,cluster_id,count,tags
0,-1,465,social reclama woman tech incident health realty culture economics auto politics sport science
1,0,921,social reclama woman tech incident health realty culture economics auto sport politics science
2,1,25,tech culture economics auto sport politics science
3,2,17,social tech culture economics auto politics
4,3,120,social woman incident tech culture economics auto politics sport science
...,...,...,...
78,77,10,incident
79,78,5,incident
80,79,3,incident
81,80,3,incident auto


In [15]:
display( data.query('cluster_id==2') )

,text,tag,cluster_id
11,"Путин велел правительству внести в Думу закон об отсрочках... и внести\nв Госдуму проект федерального закона, предусматривающего, в частности,\nпереход с 1 января 2008 года на 12-месячную военную ...",politics,2
18,"Путин поручил внести в Думу проект закона о 12-месячной военной службе...\nРФ доработать и внести в Госдуму законопроект, предусматривающий с 1\nянваря 2008 года переход на 12-месячную военную слу...",politics,2
351,"Украина предложила России компромисс по цене на газ Комитет Верховной\nРады по вопросам топливно-энергетического комплекса, ядерной политике и\nядерной безопасности предложил новый пакет по закупк...",politics,2
830,В Могилевской области появится своя резиденция Деда Мороза\n\n1 декабря 2016 в 9:27\n\nМогилевский облисполком\n\nВ охотничьем комплексе La Proni 20 декабря откроется «Чаусская резиденция Деда Мор...,social,2
1039,"С января 2017 года банки не будут ставить терминалы, которые не принимают бесконтактные карточки\n\n5 декабря 2016 в 12:13\n\nFINANCE.TUT.BY\n\nС 1 января 2017 года белорусские банки перестанут по...",economics,2
1586,"По подсчетам, в среднем он слушал по 67 композиций в день.\n\nВ Санкт-Петербурге живет поклонник творчества группы «Аукцыон», который прослушал песни коллектива 25,3 тыс. раз, а в Новосибирске — ф...",culture,2
1794,Саратовская область попала в число худших регионов по освоению средств дорожного фонда\n\nВ 2015 году дорожные фонды в регионах России были использованы не в полном объеме. Всего по стране было по...,auto,2
1799,"В Ярославской области не спешат осваивать «дорожные деньги»\n\nАудиторы Счетной палаты выявили отставание ряда регионов по освоению денежных средств, предусмотренных на строительство и реконструкц...",auto,2
1831,"Иран и Россия подписали в Тегеране меморандум о сотрудничестве в области нефти и энергетики. Документ, состоящий из 23 пунктов, был подписан в понедельник при участии заместителя министра энергети...",economics,2
2397,"Объявлен полный список номинантов на вторую после ""Оскара"" по значимости кинопремию ""Золотой глобус"". В этот раз не обошлось без приятных неожиданностей - целых двух. Неприличный и разудалый кинок...",culture,2
